In [ ]:
# Install dependencies (if not already installed)
# !pip install tensorflow --quiet

import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
import matplotlib.pyplot as plt
import numpy as np
import os
import random

# Step 1: Define dataset paths (✅ Use r'' for Windows)
base_dir = r"C:\Users\HP\Downloads\food-101\food-101\images"

# Step 2: Data preprocessing
# Split data into training & validation automatically (80/20 split)
datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2
)

train_generator = datagen.flow_from_directory(
    base_dir,
    target_size=(128, 128),
    batch_size=32,
    class_mode='categorical',
    subset='training'
)

val_generator = datagen.flow_from_directory(
    base_dir,
    target_size=(128, 128),
    batch_size=32,
    class_mode='categorical',
    subset='validation'
)

# Step 3: Define CNN model
model = Sequential([
    Conv2D(32, (3,3), activation='relu', input_shape=(128, 128, 3)),
    MaxPooling2D((2,2)),

    Conv2D(64, (3,3), activation='relu'),
    MaxPooling2D((2,2)),

    Conv2D(128, (3,3), activation='relu'),
    MaxPooling2D((2,2)),

    Flatten(),
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(101, activation='softmax')  # 101 food categories
])

# Step 4: Compile model
model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

# Step 5: Train model
history = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=10
)

# Step 6: Evaluate model
val_loss, val_acc = model.evaluate(val_generator)
print(f"\n✅ Validation Accuracy: {val_acc*100:.2f}%")

# Step 7: Plot accuracy and loss
plt.figure(figsize=(12,5))
plt.subplot(1,2,1)
plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Model Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()

plt.subplot(1,2,2)
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Model Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

# Step 8: Predict random images from validation set
class_labels = list(train_generator.class_indices.keys())
print("Total Classes:", len(class_labels))

Found 80800 images belonging to 101 classes.
Found 20200 images belonging to 101 classes.


C:\Users\HP\AppData\Roaming\Python\Python312\site-packages\keras\src\layers\convolutional\base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/10
1131/2525 ━━━━━━━━━━━━━━━━━━━━ 13:41 589ms/step - accuracy: 0.0151 - loss: 4.5995

In [ ]:
for i in range(5):
    img, label = val_generator.next()
    idx = random.randint(0, len(img) - 1)
    prediction = model.predict(np.expand_dims(img[idx], axis=0))
    predicted_class = class_labels[np.argmax(prediction)]
    
    plt.imshow(img[idx])
    plt.title(f"Predicted: {predicted_class}")
    plt.axis('off')
    plt.show()